###  Miniature Non-Uniform Illumination (NUI) Robustness Project
### CIFAR-10 (Cats vs Dogs) – Clear 20–30% Accuracy Gap Example

setup

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib
import tqdm
import sys

print("Python:", sys.version)
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("NumPy:", np.__version__)
print("Matplotlib:", matplotlib.__version__)
print("TQDM:", tqdm.__version__)


Python: 3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]
Torch: 2.9.0+cpu
Torchvision: 0.24.0+cpu
NumPy: 2.2.6
Matplotlib: 3.8.4
TQDM: 4.67.1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as T
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.rcParams['figure.figsize'] = (10, 5)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

1 Non-Uniform Illumination (NUI) Masks

In [ ]:
def generate_strong_nui_mask(h, w, strength=3.0, exponent=2.0):
    """
    Generate a strong directional uneven illumination mask.
    h, w: height and width
    strength: how strong the illumination change is
    exponent: controls the gradient sharpness
    """
    yy, xx = np.meshgrid(np.linspace(-1, 1, h), np.linspace(-1, 1, w), indexing='ij')
    angle = np.random.uniform(0, np.pi)
    grad = np.cos(angle) * xx + np.sin(angle) * yy
    grad = (grad - grad.min()) / (grad.max() - grad.min())  # normalize 0-1
    mask = grad ** exponent
    mask = 1 + strength * (mask - 0.5)                     # apply strong modulation
    mask = np.clip(mask, 0, 2).astype(np.float32)          # clip extreme values
    return mask

def apply_mask_to_tensor(img_tensor, mask):
    """
    Apply a NUI mask to a PyTorch image tensor
    """
    mask_tensor = torch.tensor(mask).unsqueeze(0)
    if mask_tensor.dim() == 3:
        mask_tensor = mask_tensor.unsqueeze(0)
    mask_tensor = torch.nn.functional.interpolate(
        mask_tensor,
        size=img_tensor.shape[1:],
        mode='bilinear',
        align_corners=False
    ).squeeze(0)
    return img_tensor * mask_tensor


2 Load CIFAR-10 Small Subset (Cats vs Dogs)

In [ ]:
transform = T.Compose([T.ToTensor()])

trainset_full = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset_full  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

classes_to_use = [3, 5]  # cat=3, dog=5
train_idx = [i for i, t in enumerate(trainset_full.targets) if t in classes_to_use][:3000]
test_idx  = [i for i, t in enumerate(testset_full.targets) if t in classes_to_use][:400]

trainset = Subset(trainset_full, train_idx)
testset  = Subset(testset_full, test_idx)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = DataLoader(testset, batch_size=64, shuffle=False)

print(f"Train samples: {len(trainset)} | Test samples: {len(testset)}")


3 Tiny CNN Model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class SqueezeNetCustom(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        # Load pretrained SqueezeNet
        self.model = models.squeezenet1_1(pretrained=False)
        
        # Replace the final classifier
        self.model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)
        self.model.num_classes = num_classes

    def forward(self, x):
        return self.model(x)


4 Train on Clean Images (Baseline)

In [ ]:
model = SqueezeNetCustom(num_classes=6).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 6
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}"):
        imgs = imgs.to(device)
        labels = torch.tensor([classes_to_use.index(l.item()) for l in labels]).to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Loss: {running_loss/len(trainloader):.4f}")
print(" Baseline training done.\n")



5 Evaluation Function

In [ ]:
def evaluate(model, loader, apply_nui=False):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            labels = torch.tensor([classes_to_use.index(l.item()) for l in labels]).to(device)
            if apply_nui:
                perturbed = []
                for img in imgs:
                    mask = generate_strong_nui_mask(
                        32, 32,
                        strength=np.random.uniform(-2.0, 2.0),
                        exponent=np.random.uniform(1.2, 4.0)
                    )
                    pert = apply_mask_to_tensor(img, mask)
                    perturbed.append(pert)
                imgs = torch.stack(perturbed)
            imgs = imgs.to(device)
            outputs = model(imgs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return correct / total


6 Baseline Evaluation (Expect ~25% drop)

In [ ]:
acc_clean = evaluate(model, testloader, apply_nui=False)
acc_nui   = evaluate(model, testloader, apply_nui=True)

print("Before Robust Training:")
print(f"Clean test accuracy: {acc_clean*100:.2f}%")
print(f"NUI test accuracy:   {acc_nui*100:.2f}%")
print("-"*40)

7 NUI-Augmented (Robust) Training

In [ ]:
def apply_nui_to_img(img_tensor):
    # img_tensor shape: (C, H, W)
    h, w = img_tensor.shape[1], img_tensor.shape[2]
    mask = generate_strong_nui_mask(h, w)
    return apply_mask_to_tensor(img_tensor, mask)

def train_model_mixed(model, trainloader, optimizer, criterion, epochs, nui_ratio=0.2):
    model.train()
    for epoch in range(epochs):
        for imgs, labels in trainloader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            # Apply NUI to a random 20% of the batch
            mask = torch.rand(len(imgs)) < nui_ratio
            imgs_aug = []
            for i, img in enumerate(imgs):
                if mask[i]:
                    imgs_aug.append(apply_nui_to_img(img))  # your NUI function
                else:
                    imgs_aug.append(img)
            imgs_aug = torch.stack(imgs_aug)

            # Forward, backward, optimize
            optimizer.zero_grad()
            outputs = model(imgs_aug)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


In [ ]:
model_nui = SqueezeNetCustom(num_classes=6).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_nui.parameters(), lr=0.001)
train_model_mixed(model_nui, trainloader, optimizer, criterion, epochs=3, nui_ratio=0.8)
print("NUI-Augmented robust training complete.\n")


8 Evaluate Robust Model

In [ ]:
acc_clean_aug = evaluate(model_nui, testloader, apply_nui=False)
acc_nui_aug   = evaluate(model_nui, testloader, apply_nui=True)

print("After NUI-Augmented Training:")
print(f"Clean test accuracy: {acc_clean_aug*100:.2f}%")
print(f"NUI test accuracy:   {acc_nui_aug*100:.2f}%")
print("-"*40)
print(f"Accuracy Drop Before: {(acc_clean - acc_nui)*100:.2f}%")
print(f"Accuracy Drop After:  {(acc_clean_aug - acc_nui_aug)*100:.2f}%")



9 Visualize Illumination Effect

In [ ]:
# Visualize single example with NUI mask
imgs, _ = next(iter(testloader))
mask = generate_strong_nui_mask(32, 32, strength=1.5, exponent=2.5)
pert = apply_mask_to_tensor(imgs[0], mask)

# Convert to numpy for display
img_np  = np.transpose(imgs[0].numpy(), (1,2,0))
mask_np = mask
pert_np = np.transpose(pert.numpy(), (1,2,0))

# Plot original, mask, and NUI-applied versions
plt.figure(figsize=(9,3))
plt.subplot(1,3,1)
plt.imshow(np.clip(img_np, 0, 1))
plt.title("Original")
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(mask_np, cmap='hot', vmin=0, vmax=2)
plt.title("Illumination Mask")
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(np.clip(pert_np, 0, 1))
plt.title("Strong NUI Applied")
plt.axis('off')

plt.tight_layout()
plt.show()

# Compute robustness improvement
drop_before = abs(acc_clean - acc_nui)*100
drop_after = abs(acc_clean_aug - acc_nui_aug)*100
improvement = drop_before - drop_after
print(f"   NUI Robustness Improvement: {improvement:.2f}%")
